In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.utils import save_image
import numpy as np
import torchvision.transforms.functional as F

In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, kernelSize = 3, inChannels = 128, outChannels = 128, strd = 1, paddng = 1):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels = inChannels, out_channels = outChannels, kernel_size = kernelSize, stride = strd, padding = paddng),
            nn.BatchNorm2d(128),
            nn.PReLU(),
            nn.Conv2d(in_channels = inChannels, out_channels = outChannels, kernel_size = kernelSize, stride = strd, padding = paddng),
            nn.BatchNorm2d(128)
        )
    def forward(self, x):
        out = self.block(x)
        return torch.add(out, x)

In [3]:
class UpsampleBlock(nn.Module):
    def __init__(self, inChannels,scaleFactor):
        super().__init__()
        self.conv = nn.Conv2d(in_channels= inChannels, out_channels= inChannels * scaleFactor ** 2, kernel_size=3, stride=1, padding=1)
        self.ps = nn.PixelShuffle(scaleFactor)
        self.act = nn.PReLU(inChannels)
    def forward(self, x):
        return self.act(self.ps(self.conv(x)))

In [4]:
class SRResnet(nn.Module):
    def __init__(self):
        super(SRResnet, self).__init__()

        self.l1 = nn.Conv2d(kernel_size=9, stride=1, in_channels=3, out_channels=128, padding=4)
        self.l2 = nn.PReLU()

        self.residuals = nn.Sequential()
        for _ in range(0, 8):
            self.residuals.add_module('residualBlock',ResidualBlock())

        self.postResiduals = nn.Sequential(
            nn.Conv2d(in_channels= 128, out_channels=128, kernel_size= 3, stride=1, padding=1),
            nn.BatchNorm2d(128),
        )
        self.upsample = UpsampleBlock(128, 2)
        #self.upsample2 = UpsampleBlock(128, 2)

        self.final = nn.Conv2d(128, 3, kernel_size= 9, stride=1, padding=4)

    def forward(self, x):
        x = self.l1(x)
        x1 = self.l2(x)
        x = self.residuals(x1)
        x = self.postResiduals(x)
        x = torch.add(x, x1)
        x = self.upsample(x)
        #x = self.upsample2(x)
        x = self.final(x)

        return x

In [5]:
# Set the path to your data folder

data_path = "dataset/"

# Define the transformations for your data
transform = transforms.Compose([
    transforms.CenterCrop((256, 256)),  # Resize the images to a fixed size
    transforms.ToTensor(),          # Convert images to tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize image pixels to the range [-1, 1]
])
transformLr = transforms.Compose([
    transforms.CenterCrop((256, 256)),
    transforms.Resize((128,128)), # Resize the images to a fixed size
    transforms.ToTensor(),          # Convert images to tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize image pixels to the range [-1, 1]
])


# Load the high-resolution and low-resolution images
hr_dataset = ImageFolder(root=data_path + "lr", transform=transform)
lr_dataset = ImageFolder(root=data_path + "lr", transform=transformLr)
#sr_dataset = ImageFolder(root=data_path + "autoencodertrain", transform=transform)

# Create the data loader for high-resolution and low-resolution images
batch_size = 15
num_workers = 2  # Set the number of worker processes for data loading
hr_data_loader = DataLoader(hr_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
lr_data_loader = DataLoader(lr_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [6]:
# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Carrega Rede
srresnet = SRResnet()
#srresnet.load_state_dict(torch.load('Model_Snapshots/Gen/Modelo_atual.pt'))

# Move the models to the device
srresnet.to(device)

content_loss = nn.MSELoss()

cuda


test_path = "Testes/"

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

test_dataset = ImageFolder(root=test_path, transform= test_transform)

batch_size = 1
num_workers = 1
test_data_loader = DataLoader(test_dataset, batch_size= batch_size, num_workers= num_workers)

i = 1
for image in test_data_loader:
    sr_images = srresnet(image[0].float().to(device))
    save_image(sr_images, f"{i}.png", normalize = True)
    i += 1

In [7]:
lr = 0.01
num_epochs = 150
save_interval = 50
arrayError = np.zeros(shape=save_interval)

In [8]:
# Training loop
for epoch in range(0, num_epochs):
    # Define the optimizers for generator and discriminator
    lr /= 5
    betas = (0.5, 0.9)
    optimizer_gen = optim.Adam(srresnet.parameters(), lr = lr, betas = betas)
    for i, (hr_images, lr_images) in enumerate(zip(hr_data_loader, lr_data_loader)):
        #factor -= step
        # Move images to the device
        hr_images = hr_images[0].to(device).float()
        lr_images = lr_images[0].to(device).float()

        srresnet.zero_grad()
        optimizer_gen.zero_grad()

        # Calculating error
        sr_images = srresnet(lr_images)
        loss = content_loss(sr_images, hr_images)
        loss.backward()
        optimizer_gen.step()

        arrayError[i % save_interval] = loss.item()

        # Print progress
        #if(i + 1) % 10 == 0:
            #print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(hr_data_loader)}]," f"Generator Loss: {loss.item():.4f}. ")
        # Print mean
        if i % save_interval == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(hr_data_loader)}], " f"Mean Loss: {arrayError.mean():.6f}.")

        if (i + 1) % save_interval == 0:
            torch.save(srresnet.state_dict(), f"Model_Snapshots/Gen/srresnet_model_epoch{epoch+1}_batch{i+1}.pt")
            print(f"Saved SRResNet model and images at epoch {epoch+1}, batch {i+1}.")

            #save_image(sr_images, f"AE-results/sr_image_epoch{epoch + 1}_batch{i+1}.png", normalize = True)
            save_image(sr_images, f"Net_Results/image_epoch{epoch + 1}_batch{i+1}_sr.png", normalize = True)
            save_image(hr_images, f"Net_Results/image_epoch{epoch + 1}_batch{i+1}_hr.png", normalize = True)
            save_image(lr_images, f"Net_Results/image_epoch{epoch + 1}_batch{i+1}_lr.png", normalize = True)

Epoch [1/150], Step [1/7886], Mean Loss: 0.007666.
Saved SRResNet model and images at epoch 1, batch 50.
Epoch [1/150], Step [51/7886], Mean Loss: 6.390251.
Saved SRResNet model and images at epoch 1, batch 100.
Epoch [1/150], Step [101/7886], Mean Loss: 28.019765.
Saved SRResNet model and images at epoch 1, batch 150.
Epoch [1/150], Step [151/7886], Mean Loss: 0.830430.
Saved SRResNet model and images at epoch 1, batch 200.
Epoch [1/150], Step [201/7886], Mean Loss: 11.285176.
Saved SRResNet model and images at epoch 1, batch 250.
Epoch [1/150], Step [251/7886], Mean Loss: 10.461325.
Saved SRResNet model and images at epoch 1, batch 300.
Epoch [1/150], Step [301/7886], Mean Loss: 0.993787.
Saved SRResNet model and images at epoch 1, batch 350.
Epoch [1/150], Step [351/7886], Mean Loss: 2.169505.
Saved SRResNet model and images at epoch 1, batch 400.
Epoch [1/150], Step [401/7886], Mean Loss: 7.735205.
Saved SRResNet model and images at epoch 1, batch 450.
Epoch [1/150], Step [451/7886

KeyboardInterrupt: 